<a href="https://colab.research.google.com/github/Hong-Soonbin/Wanted_crawling_project/blob/main/main_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#크롤링

In [562]:
import time
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from glob import glob
import warnings
import re
import os
warnings.simplefilter(action='ignore', category=FutureWarning)

## 게시물 id 수집

In [ ]:
    '''
    tag_type_list : [] ==> 직무코드별 탐색
    year : [a,b] ==> 경력 a이상b이하, [-1,-1]입력시 모든경력

    1
    0번째 게시물부터 100개씩 크롤링 while true
    오류 => ex) 총 게시물이 321개인데 300개 크롤링 후 다음100개를 크롤링하려했기때문
    따라서 재귀호출을 통해 크롤링 수를 100개씩 -> 오류 -> 10개씩 -> 오류 -> 1개씩 -> return
    만약 게시물이 321개라면 300개 크롤링 -> 20개 크롤링 - 1개 크롤링 return

    2 
    [1024, 1025, 10231, 1634, 655] # 데이터사이언티스트, 빅데이터엔지니어, DBA, 머신러닝엔지니어, 데이터엔지니어를
    쿼리에 한번에 넣으니 합집합이아닌 교집합만 출력됨
    이를 해결하기 위해 return_id_list에 list형태로 ex)[1024, 1025, 10231, 1634, 655] 값을 넣으면 각 값의 id를 모두 추출하고 중복을 제거하는
    방식으로 변경

    코드 사용 예시:
    tag_type_list = [1024, 1025, 10231, 1634, 655] # 데이터사이언티스트, 빅데이터엔지니어, DBA, 머신러닝엔지니어, 데이터엔지니어
    id_list = return_id_list(tag_type_list)
    '''

In [ ]:
#서버에 url과 쿼리로 요청
def request(url, params):
    r = requests.get(url,
                    params = params)
    r = r.json()
    return [i['id'] for i in r['data']]


# 재귀를 통해서 100자리 10자리 1자리 단위로 크롤링 반복
def get_recursive(url, params, limit, id_list=[]):
    try:
        while True:
            id_list += request(url=url, params=params)
            params['offset'] += limit
    except:
        if limit > 1:  
            params['offset'] -= limit  
            new_limit = limit / 10
            return get_recursive(url, params, new_limit, id_list)
        else:
            return id_list
    return id_list


# 모든 게시물의 id 수집
def crawl(tag_type_list=None, year=[-1,-1]):
    result = []

    url = "https://www.wanted.co.kr/api/v4/jobs?"

    params = {
    1676874787204:'',    #사용자id?
    'country': 'all',
    'job_sort': 'job.latest_order',    #최신순 정렬
    'locations': all,
    'years': year[0],    #경력 이상    경력상관없이 검색하려면 -1 , 신입은0
    'years': year[1],    #경력 이하    경력상관없이 검색하려면 -1 , 신입은0 최대10
    'limit': 100,    #한 번에 조회 가능한 수 (최대100)
    'offset': 0     #조회할 게시물의 첫 index        ex) limit=100 offset=10  => 10번게시물부터 110번게시물까지 크롤링
    }

    if tag_type_list is not None:
        for tag in tag_type_list:
            limit = 100
            offset = 0
            params['tag_type_ids'] = tag
            params['limit'] = limit
            params['offset'] = offset

            result += get_recursive(url,params,limit)   
    else:
        return get_recursive(url,params,100)   
    
    return result

In [ ]:
# 13000개 약 4분
id_list = crawl()

In [ ]:
len(id_list)

In [561]:
# 메인 크롤링 함수 id를 기반으로 게시물에 접근
def crawl_job(id_list):
    df_list = []

    for i in id_list:
        url = f'https://www.wanted.co.kr/api/v4/jobs/{i}?1691910172918'

        r = requests.get(url)
        r = r.json()['job']

        df_list.append(r)
        
    return pd.DataFrame(df_list)

In [ ]:
# 데이터 저장 경로
if not os.path.exists('data'):
    os.makedirs('data')

# 1000개씩 크롤링
for idx in range(0,len(id_list),1000):

    job = crawl_job(id_list[idx:idx+1000])
    job.to_json(f'./data/raw_{idx}_{idx+1000}.json', orient='records')

In [553]:
# 데이터 전처리

def engineering(df):
    #사용하지 않을 컬럼 drop
    drop_col = [
        'is_crossboarder', 'is_like', 'due_time', 'score', 'hidden','status','is_bookmark', 'reward', 'detail',
        'has_analysis','is_company_follow', 'compare_country','matching_score','short_link', 'like_count',
        'company'
    ]

    # 직무별 고유 id dict
    # 이번 크롤링에서는 직군을 나누지 않고 모두 수집하기 때문에 사용하지 않음
    category_tags_dict = {873:'웹 개발자', 872:'서버 개발자', 669:'프론트엔드 개발자',10110:'소프트웨어 엔제니어', 660:'자바 개발자',
                        677:'안드로이드 개발자', 678:'iOS 개발자', 895:'Node.js 개발자', 655:'데이터 엔지니어', 899:'파이썬 개발자',
                        674:'DevOps / 시스템 관리자', 900:'C,C++ 개발자', 665:'시스템,네트워크 관리자', 1634:'머신러닝 엔지니어', 1024:'데이터 사이언티스트',
                        1025:'빅데이터 엔지니어', 676:'QA,테스트 엔지니어', 877:'개발 매니저', 1026:'기술지원', 671:'보안 엔지니어',
                        876:'프로덕트 매니저', 1027:'블록체인 플랫폼 엔지니어', 893:'PHP 개발자', 658:'임베디드 개발자', 939:'웹 퍼블리셔',
                        672:'하드웨어 엔지니어', 10111:'크로스플랫폼 앱 개발자', 661:'.NET 개발자', 896:'영상,음성 엔지니어', 10231:'DBA',
                        898:'그래픽스 엔지니어', 795:'CTO', 10112:'VR 엔지니어', 10230:'ERP전문가', 894:'루비온레일즈 개발자',
                        1022:'BI엔지니어', 793:'CIO'}

    # 주소 예외처리 필요 - 수정 예정
    #df['address'].apply(lambda x : x['geo_location']['n_location'])

    # 요구사항
    df['requirements'] = df['detail'].apply(lambda x : x['requirements'])
    # 주요업무
    df['main_tasks'] = df['detail'].apply(lambda x : x['main_tasks'])
    # 소개
    df['intro'] = df['detail'].apply(lambda x : x['intro'])
    # 복지
    df['benefits'] = df['detail'].apply(lambda x : x['benefits'])
    # 우대사항
    df['preferred_points'] = df['detail'].apply(lambda x : x['preferred_points'])

    # 산업구분
    df['industry_name'] = df['company'].apply(lambda x : x['industry_name'])
    # 회사명
    df['name'] = df['company'].apply(lambda x : x['name'])
    # 지원서 응답률
    df['application_response_stats'] = df['company'].apply(lambda x : x['application_response_stats'])


    #고유 id로 게시물 url 컬럼 생성
    df['url'] = df['id'].apply(lambda x : r'https://www.wanted.co.kr/wd/' + str(x))


    # 회사 키워드
    df['company_tags'] = df['company_tags'].apply(lambda x : [i['title'] for i  in x])
    # 직무id
    df['category_tags'] = df['category_tags'].apply(lambda x : [i['id'] for i  in x])
    df['skill_tags'] = df['skill_tags'].apply(lambda x : [i['title'] for i  in x])


    # 모든 데이터가 문자열로 들어가 있어서 dict의 key를 str로 변경
    # category_tags_dict = {str(i):v for i,v in category_tags_dict.items()}


    #category_tags id 맵핑
    #df['category_tags'] = df['category_tags'].apply(lambda x : [category_tags_dict[i] for i in x])

    df = df.drop(drop_col,axis=1)

    return df.reset_index(drop=True)

In [589]:
# 1000개 단위로 저장한 데이터 병합
df = []
for path in glob('./raw_json/*'):
    with open(path) as f:
        js = json.loads(f.read())
    df += js

In [590]:
df_json = pd.DataFrame(df)
df_json.shape

(13508, 25)

In [591]:
# 전처리
preprocessed_data = engineering(df_json)

In [599]:
# 병합한 raw 데이터 저장
df_json.to_json('raw.json', orient='records')
df_json.to_csv('raw.csv', index=False)

# 병합한 raw 데이터 전처리 후 저장
preprocessed_data.to_json('preprocessed_data.json', orient='records')
preprocessed_data.to_csv('preprocessed_data.csv', index=False)

In [600]:
preprocessed_data.head()


,address,id,company_images,skill_tags,logo_img,company_tags,title_img,position,category_tags,requirements,main_tasks,intro,benefits,preferred_points,industry_name,name,application_response_stats,url
0,"{'country': '한국', 'full_location': '대구시내 공유오피스...",175747,[{'url': 'https://static.wanted.co.kr/images/c...,"[Linux, C / C++, Java, Perl, Python, Shell, 딥 ...",{'origin': 'https://static.wanted.co.kr/images...,"[연봉상위1%, 인원급성장, 51~300명, 설립10년이상, 유연근무, 건강검진, ...",{'origin': 'https://static.wanted.co.kr/images...,시스템반도체 HW Engineer (신입) (대구연구소),"[658, 672, 10110]",• 전자공학과 등 관련학과 졸업 또는 24년 2월 졸업 예정자\n• 학과 과정 외의...,• Video Codec/AI network/Image processing IP R...,"[칩스앤미디어는 어떤회사인가요?]\n""Video Technology Leader f...",• 유연근무 (Core time 운영/11:30 ~ 15:30)\n• 점심식대 지원...,• C/C++에 대한 이해\n• 석사 학위 소유자,"IT, 컨텐츠",칩스앤미디어,"{'avg_rate': 94.12, 'level': 'high', 'delayed_...",https://www.wanted.co.kr/wd/175747
1,"{'country': '한국', 'full_location': '서울특별시 강남구 ...",175744,[{'url': 'https://static.wanted.co.kr/images/c...,"[React, TypeScript, Next.js]",{'origin': 'https://static.wanted.co.kr/images...,"[누적투자100억이상, 퇴사율5%이하, 51~300명, 설립4~9년, 음료, 커피,...",{'origin': 'https://static.wanted.co.kr/images...,R.Inside Frontend Engineer,[669],"• React, TypeScript에 대한 충분한 이해와 관련 경험을 보유하신 분 ...",• 교육 현장에서 활용될 Educational Performance Monitori...,[합류하게 될 R.Inside 팀을 소개합니다]\n\n본 Frontend Engin...,최.복.동 : ‘최고의 복지는 동료다.’ 훌륭한 동료들과 즐겁게 일하며 성장할 수 ...,• 문제 해결을 위해 집요하게 매달리시는 분 \n• 배포에 Vercel을 활용해보신...,"IT, 컨텐츠",뤼이드(Riiid),"{'avg_rate': 75.66, 'level': 'normal', 'delaye...",https://www.wanted.co.kr/wd/175744
2,"{'country': '한국', 'full_location': '서울특별시 용산구 ...",175742,[{'url': 'https://static.wanted.co.kr/images/c...,[],{'origin': 'https://static.wanted.co.kr/images...,"[연봉업계평균이상, 인원급성장, 퇴사율 6~10%, 51~300명, 설립10년이상,...",{'origin': 'https://static.wanted.co.kr/images...,캐스팅,[10235],[필수 자격요건]\n• 전문학사 이상\n• 전공 무관\n• 미디어/엔터테인먼트 업계...,"[주요 업무책임] \n• 캐스팅 업무 전반 (국내/외 오디션 기획 및 진행, 온/오...","직무 Summary \n\n아티스트의 가능성을 가진 인재를 찾기 위해, 국내외 캐스...","*국가별/법인별 Benefit은 차이가 있을 수 있으며, 세부 사항은 입사 시 상세...",[선호 자격요건] \n• 외국어(영어 또는 일본어) 커뮤니케이션이 능통한 분\n• ...,"예술, 스포츠, 여가",플레디스 엔터테인먼트,"{'avg_rate': 19.42, 'level': 'very_low', 'dela...",https://www.wanted.co.kr/wd/175742
3,"{'country': '한국', 'full_location': '서초구 사평대로 3...",175739,[{'url': 'https://static.wanted.co.kr/images/c...,"[Confluence, JIRA]",{'origin': 'https://static.wanted.co.kr/images...,"[퇴사율5%이하, 50명이하, 설립10년이상, 육아휴직, 출산휴가, 음료, 간식, ...",{'origin': 'https://static.wanted.co.kr/images...,Solution Architect,"[671, 674]",- 10+ years of experience building client and ...,- Resolve technical problems as they arise\n- ...,To support our ambitions to provide an innovat...,라쿠텐심포니 코리아팀은 라쿠테니안들이 뛰어난 동료들과 함께 최고의 퍼포먼스를 낼 수...,- Comfortable to work across several teams in ...,"IT, 컨텐츠",라쿠텐심포니코리아,"{'avg_rate': 0.0, 'level': 'very_low', 'delaye...",https://www.wanted.co.kr/wd/175739
4,"{'country': '한국', 'full_location': '서울특별시 강남구 ...",175738,[{'url': 'https://static.wanted.co.kr/images/c...,"[Unreal Engine, MMO, Python, Scrum, C, C++, Ba...",{'origin': 'https://static.wanted.co.kr/images...,"[퇴사율5%이하, 301~1,000명, 설립4~9년, 워크샵, 식비, 음료, 간식,...",{'origin': 'https://static.wanted.co.kr/images...,[캐쥬얼 RPG] 게임 기획자,[892],- 문제 해결을 위한 적극적인 자세와 긍정적인 태도를 가지신 분\n- 논리적으로 사...,- 신규 개발 게임 프로젝트 참여\n- 캐주얼 RPG 컨텐츠 기획\n- 성장 및 스...,엔픽셀은 게임의 판을 바꿀 수 있는 대담한 혁신을 기반으로 만들어 내는 자사의\nA...,※ 유연한 근무환경을 지원해요!\n- 시간을 자율적으로 관리할 수 있는 유연근무제를...,- 라이브 서비스 경험이 있으신 분\n- Unity Project에 대한 경험이 있...,"IT, 컨텐츠",엔픽셀,"{'avg_rate': 76.16, 'level': 'normal', 'delaye...",https://www.wanted.co.kr/wd/175738


In [ ]:
# 요구경력 추가 수집
# 수정 전 코드 수정 예정
def year_creat(tag_type_list = [1024, 1025, 10231, 1634, 655]):
    '''
    직무list를 넣으면 for i in range(11) 
    경력을 0년 1년 ....10년 각각 크롤링해서 중복 제거 후 dict에 담아서 리턴

    0-5년차를 요구하는 게시물의id는 0,1,2,3,4,5컬럼에 모두 id 존재하는 문제가 있음(총5개의 중복 게시물)
    '''

    year_id_df = {i : set(return_id_list(tag_type_list,year=[i,i])) for i in range(11)}

    year_id_df = pd.DataFrame.from_dict(year_id_df, orient='index').T

    return year_id_df

def year_mapping(year_id_df, df):
    '''
    df['year'] = [최소경력,최대경력] 으로 구성

    중복되는 yeqr값에 동시에 존재하는 id를 줄여 연산을 최적화하기 위해서
    
    각 컬럼을 set으로 만들고 최소경력을 위한 increase변수를 보면
    1년차 = 1년차집합-0년차집합 =>0년차에 존재하지않는 1년차원소만 존재하게됨 최소경력은 0년차가 제일 중요하기 떄문
    이렇게 모든 경력을 딕셔너리 컴프리핸션으로 반복하게되면 더이상 모든 컬럼에 중복되는 id가 존재하지 않는다.

    가끔 경력요구사항이 없는 공고가 존재해서 이는 경력무관이라고 판단
    '''
    increase = {i : list(set(year_id_df.iloc[:,i]) - set(year_id_df.iloc[:,i-1])) for i in range(1,11)}
    increase[0] = list(year_id_df.iloc[:,0])
    
    decrease = {i : [set(year_id_df.iloc[:,i]) - set(year_id_df.iloc[:,i+1])] for i in range(9,-1,-1)}
    decrease[10] = list(year_id_df.iloc[:,10])

    df['year'] = [[-1,-1]] * len(df)

    for key in range(0,11):
        #increase에 존재하는 id가 있는 df 검색 후 in 한다면 경력 추가
        df.loc[df['id'].isin(increase[key]),'year'] = df.loc[df['id'].isin(increase[key]),'year'].apply(lambda x : [x[0]+1+key,x[1]] )
        df.loc[df['id'].isin(decrease[key]),'year'] = df.loc[df['id'].isin(decrease[key]),'year'].apply(lambda x : [x[0],x[1]+1+key] )
    

    return df

# year_id_df = year_creat()

# df = year_mapping(year_id_df.iloc[:,1:],df)

In [ ]:
# 수정 전 코드
# 수정 필요




# tag_type_list = [1024, #데이터사이언티스트
#                  1025, #빅데이터엔지니어
#                  10231,#DBA
#                  1634, #머신러닝엔지니어
#                  655]  #데이터엔지니어

# #조건직무에 맞는 id 크롤링
# id_list = return_id_list(tag_type_list)


# #id별 게시물에 접근에서 크롤링
# df_job = crawl_job(id_list)

# #데이터 전처리
# df = engineering(df_job)

# # year(요구경력)컬럼 추가
# year_id_dict = year_creat()
# df = year_mapping(year_id_dict,df)